In [6]:
import sqlite3, os, sys
import pandas as pd
import tensorflow as tf
import numpy as np
from tqdm.notebook import tqdm

In [7]:
# name='IC8611_oscNext_003_final'
name='dev_lvl7_mu_nu_e_classification_v003'
db_path = f"../../../../pcs557/databases/{name}/data/{name}.db"
transform_path=f"../../../../pcs557/databases/{name}/meta/transformers.pkl"
print(db_path, transform_path)

../../../../pcs557/databases/dev_lvl7_mu_nu_e_classification_v003/data/dev_lvl7_mu_nu_e_classification_v003.db ../../../../pcs557/databases/dev_lvl7_mu_nu_e_classification_v003/meta/transformers.pkl


In [8]:
transform_path='../db_files/dev_lvl7/transformers.pkl'
db_path= '../db_files/dev_lvl7/dev_lvl7_mu_nu_e_classification_v003.db'
conn = sqlite3.connect(db_path)
def get_event_no():
        print('Reading sets')
        sets = pd.read_pickle('../db_files/dev_lvl7/sets.pkl')
        train_events = sets['train']
        test_events = sets['test']
        return train_events['event_no'].to_numpy(), test_events['event_no'].to_numpy()
_, test_events=get_event_no()
df_feat=pd.read_sql(f"select * from features where event_no in {tuple(test_events)} limit 100", conn)
df_feat.head()

Reading sets


,event_no,charge_log10,dom_time,dom_x,dom_y,dom_z,width,pmt_area,rqe
0,890001,0.416667,0.250936,0.958927,-0.707658,-0.999919,1.0,0.0,-0.35
1,890001,0.666667,1.254682,0.000000,0.255849,-0.666075,1.0,0.0,0.00
2,890001,0.333333,1.956929,0.538696,1.427914,-0.935265,1.0,0.0,-0.35
3,890001,0.000000,2.271536,-0.057160,0.959408,-0.577771,0.0,0.0,0.00
4,890001,-0.166667,1.559925,-0.057160,0.959408,-0.690855,0.0,0.0,0.00


In [ ]:
features =["event_no", "dom_x", "dom_y", "dom_z", "dom_time", "charge_log10", "width", "rqe"]
feature_call = ", ".join(features)
df_feat=pd.read_sql(f"select {feature_call} from features where event_no in {tuple(test_events)}", conn)
cols=df_feat.columns

In [ ]:
transformers = pickle.load(open(transform_path, 'rb'))
trans_y      = transformers['feat']
trans_keys=trans_y.keys()

In [ ]:
df_feat.head()

In [ ]:
for col in truel[:3]:
    df_feat[col] = trans_y[col].inverse_transform(np.array(df_feat[col]).reshape(1, -1)).T

In [ ]:
df_feat=df_feat.sort(by='event_no')

In [ ]:
event_nos, n_doms, conuts = np.unique(df_feat.event_no, return_counts=True)

In [ ]:
x_not_split=np.array(df_feat)
xs = np.split(x_not_split, np.cumsum(counts)[:-1])

In [29]:
# %%timeit
n_data = 1000000
targets=['energy_log10', 'zenith', 'azimuth']
target_call = ", ".join(targets)
df_truth=pd.read_sql(f"select {target_call} from truth limit {n_data}", conn)
pid=['event_no', 'pid']
pid_call = ", ".join(pid)
df_pid=pd.read_sql(f"select {pid_call} from truth limit {n_data}", conn)

In [21]:
# df_truth1=df_truth[df_pid['pid']!=13]
# df_truth1

In [34]:
target_call

'energy_log10, zenith, azimuth'

In [23]:
# event_nos

In [31]:
# df_pid

In [32]:
# %%timeit
# event_nos=np.array(df_pid['event_no'][df_pid['pid']!=13])
event_nos=np.array(df_pid['event_no'])
str_eventnos=[str(event_no) for event_no in event_nos]
event_nocall=", ".join(str_eventnos)
df_feat_quick=pd.read_sql(f"select * from features where event_no in ({event_nocall})", conn)

1min 37s ± 3.08 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
event_nocall

NameError: name 'event_nocall' is not defined

In [ ]:
stop_id.fetchall()

In [ ]:
start, stop = 0, 100000
# start_id  = conn.execute(f"select distinct event_no from features where pid = 13 limit ")
event_nos = pd.read_sql(f"select distinct event_no from features limit {stop} offset {start}", conn)

In [ ]:
event_nos

In [ ]:
start, stop = 0, 100000
select_rows = ["dom_x", "dom_y", "dom_z", "time", "charge_log10"]

start_id = conn.execute(f"select distinct event_no from features limit 1 offset {start}").fetchall()[0][0]
stop_id  = conn.execute(f"select distinct event_no from features limit 1 offset {stop}").fetchall()[0][0]

In [ ]:
features = ["dom_x", "dom_y", "dom_z", "time", "charge_log10"]
targets  = ["energy_log10", "position_x", "position_y", "position_z", "direction_x", "direction_y", "direction_z"]
feature_call = ", ".join(features)
target_call  = ", ".join(targets)
df_event = pd.read_sql(f"select event_no       from features where event_no >= {start_id} and event_no < {stop_id}", conn)
df_feat  = pd.read_sql(f"select {feature_call} from features where event_no >= {start_id} and event_no < {stop_id}", conn)
df_targ  = pd.read_sql(f"select {target_call } from truth    where event_no >= {start_id} and event_no < {stop_id}", conn)

In [ ]:
idx_list    = np.array(df_event)
x_not_split = np.array(df_feat)

In [ ]:
np.shape(idx_list)

In [ ]:
ids, idx, counts = np.unique(idx_list.flatten(), return_index = True, return_counts = True) 

In [ ]:
np.sum(counts)

In [ ]:
x_not_split.shape

In [ ]:
xs = np.split(x_not_split, np.cumsum(counts))

In [ ]:
ys = np.array(df_targ)

In [ ]:
np.shape(xs)

In [ ]:
for x, y in zip(xs, ys):
    print(x, y)
    break

In [ ]:
from sklearn.neighbors import kneighbors_graph as knn
from spektral.data import Graph
neighbors=6

In [ ]:
print("Generating adjacency matrices")
graph_list = []
for x, y in tqdm(zip(xs, ys), total = len(xs)):
    a = knn(x[:, :3], neighbors)



    graph_list.append(Graph(x = x, a = a, y = y))

graph_list = np.array(graph_list, dtype = object)

In [ ]:
s = [x.shape[0] for x in xs]

In [ ]:
xs = np.array(xs, dtype = object)[np.argsort(s)]

In [ ]:
size, idx = np.unique(np.sort(s), return_index = True)

In [ ]:
preprocess(list(xs[4: 9]))

In [ ]:
np.sort(s)

In [ ]:
idx

In [ ]:
import pickle
transformers = pickle.load(open('../db_files/muongun/transformers.pkl', 'rb'))
trans_x      = transformers['features']
trans_y      = transformers['truth']

In [15]:
diff={'hyper_params':{'hidden_states': [128,64,32],
     'dropout': [0,0.05],
     'conv_layers': [1,2,3],
     'decode_layers': [2,4]},
     'run_params':{'batch_size':[512,1024,2048]}}

In [23]:
diff={'hidden_states': [128,64,32],
     'dropout': [0,0.05],
     'conv_layers': [1,2,3],
     'decode_layers': [2,4],
     'batch_size':[512,1024,2048]}

In [42]:
def perms(diffs):
    from itertools import product
    keys=list(diffs.keys())
    val=list(diffs.values())
    for i, s in enumerate(val):
        if i==0:
            a=val[0]
        else:
            a=product(a, val[i])
    bs=[]
    for b in a:
        bs.append(b)
    output=[]
    def removeNestings(l):
        for i in l:
            if type(i) == tuple:
                removeNestings(i)
            else:
                output.append(i)
    removeNestings(bs)
    perms=np.array(output)
#     perms=[int(num) if float(num).is_integer() else num for num in perms]
    return perms.reshape(-1, len(keys))

In [47]:
keys=list(diff.keys())
for j, key in enumerate(keys):
    if key in base['run_params']:
        base['run_params'][key]=p[0][j]
    elif key in base['hyper_params']:
        typ=type(base['hyper_params'][key])
        base['hyper_params'][key]=typ(p[0][j])
    elif key in base['data_params']:
        base['data_params'][key]=p[0][j]

<class 'int'>
<class 'float'>
<class 'int'>
<class 'int'>


In [44]:
p=perms(diff)

In [38]:
p

array([[1.280e+02, 0.000e+00, 1.000e+00, 2.000e+00, 5.120e+02],
       [1.280e+02, 0.000e+00, 1.000e+00, 2.000e+00, 1.024e+03],
       [1.280e+02, 0.000e+00, 1.000e+00, 2.000e+00, 2.048e+03],
       [1.280e+02, 0.000e+00, 1.000e+00, 4.000e+00, 5.120e+02],
       [1.280e+02, 0.000e+00, 1.000e+00, 4.000e+00, 1.024e+03],
       [1.280e+02, 0.000e+00, 1.000e+00, 4.000e+00, 2.048e+03],
       [1.280e+02, 0.000e+00, 2.000e+00, 2.000e+00, 5.120e+02],
       [1.280e+02, 0.000e+00, 2.000e+00, 2.000e+00, 1.024e+03],
       [1.280e+02, 0.000e+00, 2.000e+00, 2.000e+00, 2.048e+03],
       [1.280e+02, 0.000e+00, 2.000e+00, 4.000e+00, 5.120e+02],
       [1.280e+02, 0.000e+00, 2.000e+00, 4.000e+00, 1.024e+03],
       [1.280e+02, 0.000e+00, 2.000e+00, 4.000e+00, 2.048e+03],
       [1.280e+02, 0.000e+00, 3.000e+00, 2.000e+00, 5.120e+02],
       [1.280e+02, 0.000e+00, 3.000e+00, 2.000e+00, 1.024e+03],
       [1.280e+02, 0.000e+00, 3.000e+00, 2.000e+00, 2.048e+03],
       [1.280e+02, 0.000e+00, 3.000e+00,

In [31]:
print([int(num) if float(num).is_integer() else num for num in p])

TypeError: only size-1 arrays can be converted to Python scalars

In [46]:
true, false=1,0
base={
    "experiment": "IceCube",
    "group":  "compare", 

    "model_name":  "Sage",
    "angle_unitvec": "angle",
    "wandblog": true,

    "run_params":{
    "epochs":     20,
    "batch_size": 512,
    "early_stop": true,
    "zeniazi_metric":true,
    "patience":   5,
    "lr_schedule": "classic",
    "learning_rate":   1e-3,
    "warm_up": 3,
    "lr_decay": 0.9,
    "val_epoch":   10,
    "loss_func": "abs_vonMises2D_angle",
    "metrics": "energy_angle_zeniazi",
    "performance_plot": "performance_vM2D"
},

    "hyper_params": {
        "n_out":           3,
        "n_sigs":          2,
        "hidden_states":   64,
        "glob": true,
        "conv_layers":   2,
        "conv_activation": "relu",
        "decode_layers":   3,
        "decode_activation": 1,
        "dropout":         0.05,
        "regularization": "l2", 
        "batch_norm":      true,
        "forward":  true,
        "edgeconv": 0,
        "edgenorm":0
    },

   "data_params":{ 
    "n_data": 2e4,
    "graph_construction":       "classic",
    "muon":             true,
    "n_neighbors":       6,
    "data_split":      [0.8, 0.1, 0.1],
    "restart":   false,
    "transform_path": "../db_files/muongun/transformers.pkl",
    "db_path": "../db_files/muongun/rasmus_classification_muon_3neutrino_3mio.db",
    "features":   ["dom_x", "dom_y", "dom_z", "time", "charge_log10", "SRTInIcePulses", "lc"],
    "targets":    ["energy_log10", "zenith","azimuth"],
    "skip": 0,
    "SRT": 0,
    "database": "MuonGun"
}
}

In [ ]:
l=bs
outp
  
# Driver code
print ('The original list: ', l)
removeNestings(l)
print ('The list after removing nesting: ', output)

In [ ]:
perms=np.array(output)
perms.reshape(-1, len(keys))

In [ ]:
bs=np.array(bs)

In [ ]:
bs.reshape(len(keys), len(bs))